# Demo Notebook

Steps:
1. Download SAE with SAE Lens.
2. Create a dataset consistent with that SAE. 
3. Fold the SAE decoder norm weights so that feature activations are "correct".
4. Estimate the activation normalization constant if needed, and fold it into the SAE weights.
5. Run the SAE generator for the features you want.

# Set Up

In [1]:
import json
import torch
from sae_lens import SAE
from transformer_lens import HookedTransformer
from sae_dashboard.sae_vis_data import SaeVisConfig
from sae_dashboard.sae_vis_runner import SaeVisRunner

## Step 1. Download / Initialize SAE

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")
model = HookedTransformer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct", device=device, dtype="bfloat16")
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="llama-3.2-1B-mlp-math",  # see other options in sae_lens/pretrained_saes.yaml
    sae_id="blocks.8.hook_mlp_out",  # won't always be a hook point
    device=device,
)
# fold w_dec norm so feature activations are accurate
sae.fold_W_dec_norm()


Device: cuda
Loading model config for meta-llama/Llama-3.2-1B-Instruct
Loaded model config for {'d_model': 2048, 'd_head': 64, 'n_heads': 32, 'd_mlp': 8192, 'n_layers': 16, 'n_ctx': 2048, 'eps': 1e-05, 'd_vocab': 128256, 'act_fn': 'silu', 'n_key_value_heads': 8, 'normalization_type': 'RMS', 'positional_embedding_type': 'rotary', 'rotary_adjacent_pairs': False, 'rotary_dim': 64, 'final_rms': True, 'gated_mlp': True, 'original_architecture': 'LlamaForCausalLM', 'tokenizer_name': 'meta-llama/Llama-3.2-1B-Instruct'}


Loaded pretrained model meta-llama/Llama-3.2-1B-Instruct into HookedTransformer


/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/sae_lens/sae.py:133: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(
/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/sae_lens/sae.py:536: UserWarning: norm_scaling_factor not found for llama-3.2-1B-mlp-math and blocks.8.hook_mlp_out, but normalize_activations is 'expected_average_only_in'. Skipping normalization folding.
  warnings.warn(


# 2. Get token dataset

In [4]:
from sae_lens import ActivationsStore
activations_store = ActivationsStore.from_sae(
    model=model,
    sae=sae,
    streaming=True,
    dataset="lighteval/MATH",
    store_batch_size_prompts=16,
    n_batches_in_buffer=8,
    device=device,
)
# Some SAEs will require we estimate the activation norm and fold it into the weights. This is easy with SAE Lens.
if sae.cfg.normalize_activations == "expected_average_only_in":
    norm_scaling_factor = activations_store.estimate_norm_scaling_factor(
        n_batches_for_norm_estimate=30
    )
    sae.fold_activation_norm_scaling_factor(norm_scaling_factor)

/datadrive5/huypn16/anaconda3/envs/ana/lib/python3.11/site-packages/sae_lens/training/activations_store.py:265: UserWarning: Dataset is not tokenized. Pre-tokenizing will improve performance and allows for more control over special tokens. See https://jbloomaus.github.io/SAELens/training_saes/#pretokenizing-datasets for more info.
  warnings.warn(


Token columns: ['problem']


Estimating norm scaling factor: 100%|██████████| 30/30 [00:08<00:00,  3.67it/s]


In [5]:
from tqdm import tqdm
def get_tokens(
    activations_store: ActivationsStore,
    n_prompts: int,
):
    all_tokens_list = []
    pbar = tqdm(range(n_prompts))
    for _ in pbar:
        batch_tokens = activations_store.get_batch_tokens()
        batch_tokens = batch_tokens[torch.randperm(batch_tokens.shape[0])][
            : batch_tokens.shape[0]
        ]
        all_tokens_list.append(batch_tokens)

    all_tokens = torch.cat(all_tokens_list, dim=0)
    all_tokens = all_tokens[torch.randperm(all_tokens.shape[0])]
    return all_tokens

# 1000 prompts is plenty for a demo.
token_dataset = get_tokens(activations_store, 1024)

100%|██████████| 1024/1024 [01:53<00:00,  9.03it/s]


# Step 3 Evaluate the SAE

In [13]:
from sae_lens import run_evals
from sae_lens.evals import get_eval_everything_config

eval_metrics = run_evals(
    sae=sae,
    activation_store=activations_store,
    model=model,
    eval_config=get_eval_everything_config(
        batch_size_prompts=8,
        n_eval_reconstruction_batches=10,
        n_eval_sparsity_variance_batches=3,
    )
)
print(json.dumps(eval_metrics, indent=4))
# CE Loss score should be high for residual stream SAEs
print(eval_metrics["metrics/ce_loss_score"])
# ce loss without SAE should be fairly low < 3.5 suggesting the Model is being run correctly
print(eval_metrics["metrics/ce_loss_without_sae"])
# ce loss with SAE shouldn't be massively higher
print(eval_metrics["metrics/ce_loss_with_sae"])

{
    "metrics/kl_div_with_sae": 0.06201171875,
    "metrics/kl_div_with_ablation": 0.333984375,
    "metrics/ce_loss_with_sae": 1.9921875,
    "metrics/ce_loss_without_sae": 1.9609375,
    "metrics/ce_loss_with_ablation": 2.25,
    "metrics/kl_div_score": 0.814327485380117,
    "metrics/ce_loss_score": 0.8918918918918919,
    "metrics/l2_norm_in": 3.28125,
    "metrics/l2_norm_out": 2.46875,
    "metrics/l2_ratio": 0.75,
    "metrics/l0": 1366.159423828125,
    "metrics/l1": 229.0,
    "metrics/explained_variance": 0.6875,
    "metrics/mse": 0.0007476806640625,
    "metrics/total_tokens_evaluated": 40960
}
{'metrics/kl_div_with_sae': 0.06201171875, 'metrics/kl_div_with_ablation': 0.333984375, 'metrics/ce_loss_with_sae': 1.9921875, 'metrics/ce_loss_without_sae': 1.9609375, 'metrics/ce_loss_with_ablation': 2.25, 'metrics/kl_div_score': 0.814327485380117, 'metrics/ce_loss_score': 0.8918918918918919, 'metrics/l2_norm_in': 3.28125, 'metrics/l2_norm_out': 2.46875, 'metrics/l2_ratio': 0.75, 

# 4. Generate Feature Dashboards

In [6]:
import gc

gc.collect()
torch.cuda.empty_cache()

In [21]:
from pathlib import Path

test_feature_idx_llama = list([2705, 9766, 18472, 22648, 24905, 25939, 27169, 27353, 27368, 32379])

feature_vis_config_llama = SaeVisConfig(
    hook_point=sae.cfg.hook_name,
    features=test_feature_idx_llama,
    minibatch_size_features=10,
    minibatch_size_tokens=256,  # this is number of prompts at a time.
    verbose=True,
    device="cuda",
    cache_dir=Path(
        "llama_original.layers.8_bs=256_nrows=16384"
    ),  # this will enable us to skip running the model for subsequent features.
    dtype="bfloat16",
)

data = SaeVisRunner(feature_vis_config_llama).run(
    encoder=sae,  # type: ignore
    model=model,
    tokens=token_dataset[:16384], # 16384=1024 * 16
)

Forward passes to cache data for vis:   0%|          | 0/8 [00:00<?, ?it/s]

Extracting vis data from cached data:   0%|          | 0/1 [00:00<?, ?it/s]

┏━━━━━━┳━━━━━━┳━━━━━━━┓
┃ Task ┃ Time ┃ Pct % ┃
┡━━━━━━╇━━━━━━╇━━━━━━━┩
└──────┴──────┴───────┘

In [22]:
from sae_dashboard.data_writing_fns import save_feature_centric_vis

filename = f"demo_feature_dashboards.html"
save_feature_centric_vis(sae_vis_data=data, filename=filename)

Saving feature-centric vis:   0%|          | 0/1 [00:00<?, ?it/s]